In [1]:
from hypex.dataset import Dataset, ExperimentData, InfoRole, TreatmentRole, TargetRole
from hypex.experiments.aa import AA_TEST
from hypex.reporters import AADictReporter

ModuleNotFoundError: No module named 'hypex'

In [ ]:
data = Dataset(
    roles={
        "user_id": InfoRole(int),
        "treat": TreatmentRole(int),
        "pre_spends": TargetRole(),
        "post_spends": TargetRole()
    }, data="data.csv",
)
data

In [ ]:
data.data.dtypes

In [ ]:
test = AA_TEST
ed = ExperimentData(data)
result = test.execute(ed)

In [ ]:
AADictReporter().report(result)

In [ ]:
result.analysis_tables

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

from hypex.comparators import TTest, KSTest, GroupSizes, GroupDifference
from hypex.utils import SpaceEnum
from hypex.splitters import AASplitter
from hypex.analyzers import OneAASplitAnalyzer
from hypex.experiments import CycledExperiment, Experiment, OnRoleExperiment

aa = Experiment(
    executors=[
        AASplitter(),
        OnRoleExperiment(
            executors=[
                GroupSizes(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                GroupDifference(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                TTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
                KSTest(grouping_role=TreatmentRole(), space=SpaceEnum.additional),
            ],
            role=TargetRole(),
        )
    ]
)
experiment = CycledExperiment(n_iterations=20, inner_executor=aa, analyzer=OneAASplitAnalyzer())
ed = ExperimentData(data)
res = experiment.execute(ed)

In [ ]:
res.additional_fields

In [ ]:
res.analysis_tables

In [ ]:
res_analyzer = OneAASplitAnalyzer().execute(res)

In [ ]:
res_analyzer.analysis_tables['OneAASplitAnalyzer╰╰']